In [1]:
import sklearn
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as ss
import scipy.spatial.distance as sdist

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

%matplotlib inline

In [2]:
product_df = pd.read_csv("product2.csv", sep=';', encoding='latin')
package_df = pd.read_csv("package2.csv", sep=';', encoding='latin')

In [3]:
def table_completeness(table):
    
    nb_m = table.isnull().sum().sort_values()
    ratio_m = (table.isnull().sum() / table.shape[0]).sort_values()
    manquant = pd.concat([nb_m, ratio_m], axis=1, sort=False)
    return pd.DataFrame({'Types': table[list(manquant.index.values)].dtypes,
                          'Nb Unique': [len(pd.Categorical(table[_]).categories) for _
                                        in table[list(manquant.index.values)].columns],
                          'Nb manquants': nb_m,
                          'Ratio manquants%': ratio_m,})

In [4]:
def cramers_v(x, y):
    """
        ref: https://towardsdatascience.com/the-search-for-categorical-correlation-a1cf7f1888c9
    """
    confusion_matrix = pd.crosstab(x,y)
    chi2 = ss.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2/n
    r, k = confusion_matrix.shape
    phi2corr = max(0, phi2-((k-1)*(r-1))/(n-1))
    rcorr = r-((r-1)**2)/(n-1)
    kcorr = k-((k-1)**2)/(n-1)
    return np.sqrt(phi2corr/min((kcorr-1),(rcorr-1)))

# 1.

Auscultez les données et présentez un résumé de votre auscultation (nombre d’attributs
pour chaque table, types d’attributs, valeurs manquantes, incohérences intra-attribut, incohérences inter-attribut entre attributs reliés, vraissemblance et interprétabilité des attributs) ;


## product2.csv --> product_df

File Notes
* Package data can be found in the Packages file, linked by the ProductID field.
* Reference code names (translations) are included instead of the codes themselves.
* Fields that have multiple values are identified with an “MV” after their name. Values are concatenated together by a semi-colon “;”.
* If the term NULL appears after an element name, it means there may be records where no value is provided.
* Complete list: www.fda.gov/edrls

On remarque que la colonne PRODUCTID présente 1560 valeurs manquantes.

La colonne PRODUCTNDC quant à elle présente certaines valeurs aberrantes:

In [5]:
# %%
print(product_df['PRODUCTNDC'][159:161])

159    05-juin
160    19-juin
Name: PRODUCTNDC, dtype: object


La colonne PRODUCTTYPENAME contient 7 modalités (valeurs uniques) d'une variable qualitative catégorielle. Elle n'a
pas de valeurs absente, ce qui facilitera sa numérisation.

La colonne PROPRIETARYNAME est associée à une variable qualitative textuelle avec un nombre élevé de modalités.
De plus, la longueur des valeurs (le nombre de caractères) varient également beaucoup puisqu'il s'agit des noms commerciaux
des produits.

La colonne PROPRIETARYNAMESUFFIX est du même type que PROPRIETARYNAME, mais beaucoups de valeurs sont absentes. Elle présente
diverses informations supplémentaires aux objets, et selon la documentation de la FDA il n'y a pas de standard.

La colonne NONPROPRIETARYNAME contient seulement 4 valeurs absentes. Le nombre de modalités y est très élevé. Il s'agit
de listes des ingrédients médicinaux des produits. Le formatage des listes n'est pas standard. Les 4 valeurs manquantes
semblent difficil à inférer à première vue.

La colonne DOSAGEFORMNAME contient des informations sur les modes d'utilisation du produit. L'information pourrait y être
simplifiée car elle est trop détaillée pour nos besoins. On pourrait ne conserver que le mode principale d'administration
(orale, nasal, etc.) des produit

La colonne ROUTENAME contient des données du standard de la FDA. Chaque produit présente plusieurs valeurs
séparées par un point-virgule. Il y a un nombre élevé de valeurs absentes qui, à première vue, seront difficiles à inférer.

Les colonnes STARTMARKETINGDATE et ENDMARKETINGDATE présente la date de mise en marché et de retrait du marché respectivement.
Elles sont semblables à la colonne correspondante de la table "package" à ceci près qu'il n'y à pas de valeurs absentes pour la
colonne STARTMARKETINGDATE.

La colonne MARKETINGCATEGORYNAME contient des données du standard de la FDA. Elle a 10 modalités et aucune valeur absente,
ce qui en fera une colonne facile à numériser.

La colonne APPLICATIONNUMBER présente pour chaque produit un numéro associé à sa catégorie de marketing qui se retrouve
dans la colonne MARKETINGCATEGORYNAME. Elle contient par ailleurs beaucoup de valeurs absentes.

La colonne LABELERNAME contient des données qualitatives très disparates. Elle contient par aillerus environ 550 valeurs absentes
qui semble a priori impossibles à inférer.

La colonne SUBSTANCENAME contient des données du standard de la FDA qui présente 108227 modalités. Chaque produit peut
appartenir à plusieurs catégories. La quantité de valeurs absentes est elevées et elles seront difficilement complétables.

Les colonnes ACTIVE_NUMERATOR_STRENGTH et ACTIVE_INGRED_UNIT présentent respectivement des information posologiques et des
informations sur les ingrédients médicinaux. Les valeurs sont sonvent des listes dont les éléments sont séparés par un point-virgule.
Ces colonnes présentent également un nombre égale de valeurs absentes.

La colonne PHARM_CLASSES contiennent des données du standard de la FDA. Il s'agit de du mode d'action biochimique des
produits. Les valeurs sont souvent des listes dont les éléments sont séparés par une virgule. Le nombre de valeurs
absentes y est de loin supérieur aux données présentes.

La colonne DEASCHEDULE montre la classification légale telle qu'ériger par la DEA (Drug Enforcement Agency). Il s'agit
d'une variable catégorique cardinale utilisant des chiffres romains. Il y a un grand nombre de données manquantes qui seront
difficilement complétables sans l'apport d'un spécialiste juridique.

La colonne NDC_EXCLUDE_FLAG contient une seule valeure (N). Selon la documentation disponible, cette colonne indique si le
produit a été exclu suite à une non-conformité suite à une requête faite par la FDA, la valeur N indiquant donc "No", pour
signigier que le produit n'a pas été exclu. Il n'y a aucune valeur absente.

La colonne LISTING_RECORD_CERTIFIED_THROUGH indique la date jusqu'à la laquelle l'entrée dans la table est valide, à moins
d'être mise à jour ou certifié. Il y a plusieurs valeurs absentes.

### Remarques

Nous devrons possiblement joindre les bases de données sur **ProductID**

Il est à noter que tous produits ayant une valeur à la variable **EndMarketingDate** présente une valeur NaN à la variable **Listing_Record_Certified_Through**

**NDC_Exclude_Flag** semble non essentiel puisque nous traitons que de ceux qui sont approuvés.

________

In [6]:
print("product_df shape: ", product_df.shape)
print("# of rows: ", product_df.shape[0])
print("# of attributes: ", product_df.shape[1])

product_df shape:  (93238, 20)
# of rows:  93238
# of attributes:  20


In [7]:
product_df.head()

,PRODUCTID,PRODUCTNDC,PRODUCTTYPENAME,PROPRIETARYNAME,PROPRIETARYNAMESUFFIX,NONPROPRIETARYNAME,DOSAGEFORMNAME,ROUTENAME,STARTMARKETINGDATE,ENDMARKETINGDATE,MARKETINGCATEGORYNAME,APPLICATIONNUMBER,LABELERNAME,SUBSTANCENAME,ACTIVE_NUMERATOR_STRENGTH,ACTIVE_INGRED_UNIT,PHARM_CLASSES,DEASCHEDULE,NDC_EXCLUDE_FLAG,LISTING_RECORD_CERTIFIED_THROUGH
0,NaN,0002-0800,HUMAN OTC DRUG,Sterile Diluent,NaN,diluent,"INJECTION, SOLUTION",SUBCUTANEOUS,19870710,NaN,NDA,NDA018781,10,WATER,1,mL/mL,NaN,NaN,N,20201231.0
1,NaN,0002-1200,HUMAN PRESCRIPTION DRUG,Amyvid,NaN,Florbetapir F 18,"INJECTION, SOLUTION",INTRAVENOUS,20120601,NaN,NDA,NDA202008,10,FLORBETAPIR F-18,51,mCi/mL,"Radioactive Diagnostic Agent [EPC],Positron Em...",NaN,N,20211231.0
2,NaN,0002-1433,HUMAN PRESCRIPTION DRUG,Trulicity,NaN,Dulaglutide,"INJECTION, SOLUTION",SUBCUTANEOUS,20140918,NaN,BLA,BLA125469,10,DULAGLUTIDE,.75,mg/.5mL,"GLP-1 Receptor Agonist [EPC],Glucagon-Like Pep...",NaN,N,20201231.0
3,NaN,0002-1434,HUMAN PRESCRIPTION DRUG,Trulicity,NaN,Dulaglutide,"INJECTION, SOLUTION",SUBCUTANEOUS,20140918,NaN,BLA,BLA125469,10,DULAGLUTIDE,1.5,mg/.5mL,"GLP-1 Receptor Agonist [EPC],Glucagon-Like Pep...",NaN,N,20201231.0
4,NaN,0002-1436,HUMAN PRESCRIPTION DRUG,EMGALITY,NaN,galcanezumab,"INJECTION, SOLUTION",SUBCUTANEOUS,20180927,NaN,BLA,BLA761063,10,GALCANEZUMAB,120,mg/mL,NaN,NaN,N,20201231.0


In [8]:
product_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93238 entries, 0 to 93237
Data columns (total 20 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   PRODUCTID                         91678 non-null  object 
 1   PRODUCTNDC                        93238 non-null  object 
 2   PRODUCTTYPENAME                   93238 non-null  object 
 3   PROPRIETARYNAME                   93232 non-null  object 
 4   PROPRIETARYNAMESUFFIX             10163 non-null  object 
 5   NONPROPRIETARYNAME                93234 non-null  object 
 6   DOSAGEFORMNAME                    93238 non-null  object 
 7   ROUTENAME                         91306 non-null  object 
 8   STARTMARKETINGDATE                93238 non-null  int64  
 9   ENDMARKETINGDATE                  4323 non-null   float64
 10  MARKETINGCATEGORYNAME             93238 non-null  object 
 11  APPLICATIONNUMBER                 80141 non-null  object 
 12  LABE

In [9]:
product_df.select_dtypes(include='float64').describe()

,ENDMARKETINGDATE,LISTING_RECORD_CERTIFIED_THROUGH
count,4.323000e+03,8.891300e+04
mean,2.021040e+07,2.020247e+07
std,1.539891e+05,3.294780e+03
min,2.020022e+07,2.020123e+07
25%,2.020081e+07,2.020123e+07
50%,2.020123e+07,2.020123e+07
75%,2.021111e+07,2.020123e+07
max,3.031021e+07,2.021123e+07


In [10]:
product_df.select_dtypes(include='int64').describe()

,STARTMARKETINGDATE
count,9.323800e+04
mean,2.010696e+07
std,1.131769e+05
min,1.900010e+07
25%,2.009090e+07
50%,2.014122e+07
75%,2.018010e+07
max,2.020021e+07


In [11]:
product_df.select_dtypes(include='object').describe()

,PRODUCTID,PRODUCTNDC,PRODUCTTYPENAME,PROPRIETARYNAME,PROPRIETARYNAMESUFFIX,NONPROPRIETARYNAME,DOSAGEFORMNAME,ROUTENAME,MARKETINGCATEGORYNAME,APPLICATIONNUMBER,LABELERNAME,SUBSTANCENAME,ACTIVE_NUMERATOR_STRENGTH,ACTIVE_INGRED_UNIT,PHARM_CLASSES,DEASCHEDULE,NDC_EXCLUDE_FLAG
count,91678,93238,93238,93232,10163,93234,93238,91306,93238,80141,93238,90929,90929,90929,42254,4423,93238
unique,91678,91468,7,32716,4022,16257,134,180,10,10711,6611,8976,8769,2391,1285,4,1
top,0220-3816_875ac319-f27a-4cc8-e053-2995a90ad92a,OTC MONOGRAPH FINAL,HUMAN OTC DRUG,Ibuprofen,Maximum Strength,Ibuprofen,TABLET,ORAL,ANDA,part352,REMEDYREPACK INC.,ALCOHOL,10,mg/1,"Corticosteroid [EPC],Corticosteroid Hormone Re...",CII,N
freq,1,111,46172,565,408,947,15442,54704,37490,6563,2201,1720,4617,34831,1420,1802,93238


#### Nombre, taux et types des valeurs manquantes par attribut (product_df)

In [12]:
table_completeness(product_df)

,Types,Nb Unique,Nb manquants,Ratio manquants%
PRODUCTNDC,object,91468,0,0.000000
PRODUCTTYPENAME,object,7,0,0.000000
DOSAGEFORMNAME,object,134,0,0.000000
STARTMARKETINGDATE,int64,7262,0,0.000000
NDC_EXCLUDE_FLAG,object,1,0,0.000000
MARKETINGCATEGORYNAME,object,10,0,0.000000
LABELERNAME,object,6611,0,0.000000
NONPROPRIETARYNAME,object,16257,4,0.000043
PROPRIETARYNAME,object,32716,6,0.000064
PRODUCTID,object,91678,1560,0.016731


___________________________

## package2.csv --> package_df

La colonne PRODUCTID contient la concaténation des valeurs du code de produit NDC et de l'ID SPL. Il n'y a aucune valeur
absente.

La colonne PRODUCTNDC contient cependant 1500 valeurs manquantes. On y retrouve également des valeurs aberrantes.

Les colonnes STARTMARKETINGDATE et ENDMARKETINGDATE contiennent plusieurs valeurs absentes. Elle contiennet, comme leurs noms
l'indique, des dates de mise et de retrait du marché.

La colonne PACKAGEDESCRIPTION est présentée sous forme de phrase et contient de multiples informations: le type de
volume, sa valeur et son unité. S'il existe plusieurs contenants pour un objet, ils sont concaténés par un séparateur
'>' de manière hiérarchique.

La colonne PACKAGEDESCRIPTION contient des données sous forme de texte décrivant plusieurs attributs de l'emballage
pour chaque produit. Certaines valeurs sont des "listes" hierarchique indiquant l'agencement de l'emballage pour les
produits concernés.

Les colonnes NDC_EXCLUDE_FLAG et SAMPLE_PACKAGE contiennent une variable binaire dont les valeurs dont très débalancées.
Elles indiquent l'exclusion et s'il s'agit d'un format échantillon pour le produit. Il n'y manque aucune valeur.

### Remarques

**ProductNDC** est un sous-ensemble de **NDCPackageCode**

**NDC_Exclude_Flag** semble non essentiel puisque nous traitons que de ceux qui sont approuvés.

___________________________

In [13]:
print("package_df shape: ", package_df.shape)
print("# of rows: ", package_df.shape[0])
print("# of attributes: ", package_df.shape[1])

package_df shape:  (173887, 8)
# of rows:  173887
# of attributes:  8


In [14]:
package_df.head()

,PRODUCTID,PRODUCTNDC,NDCPACKAGECODE,PACKAGEDESCRIPTION,STARTMARKETINGDATE,ENDMARKETINGDATE,NDC_EXCLUDE_FLAG,SAMPLE_PACKAGE
0,0002-0800_94c48759-29bb-402d-afff-9a713be11f0e,0002-0800,0002-0800-01,1 VIAL in 1 CARTON (0002-0800-01) > 10 mL in ...,19870710,NaN,N,N
1,0002-1200_35551a38-7a8d-43b8-8abd-f6cb7549e932,0002-1200,0002-1200-30,"1 VIAL, MULTI-DOSE in 1 CAN (0002-1200-30) > ...",20120601,NaN,N,N
2,0002-1200_35551a38-7a8d-43b8-8abd-f6cb7549e932,0002-1200,0002-1200-50,"1 VIAL, MULTI-DOSE in 1 CAN (0002-1200-50) > ...",20120601,NaN,N,N
3,0002-1433_42a80046-fd68-4b80-819c-a443b7816edb,0002-1433,0002-1433-61,2 SYRINGE in 1 CARTON (0002-1433-61) > .5 mL ...,20141107,NaN,N,Y
4,0002-1433_42a80046-fd68-4b80-819c-a443b7816edb,0002-1433,0002-1433-80,4 SYRINGE in 1 CARTON (0002-1433-80) > .5 mL ...,20141107,NaN,N,N


In [15]:
package_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173887 entries, 0 to 173886
Data columns (total 8 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   PRODUCTID           173887 non-null  object 
 1   PRODUCTNDC          172387 non-null  object 
 2   NDCPACKAGECODE      171541 non-null  object 
 3   PACKAGEDESCRIPTION  173887 non-null  object 
 4   STARTMARKETINGDATE  173887 non-null  int64  
 5   ENDMARKETINGDATE    6456 non-null    float64
 6   NDC_EXCLUDE_FLAG    173887 non-null  object 
 7   SAMPLE_PACKAGE      173887 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 10.6+ MB


In [16]:
package_df.select_dtypes(include=['float64']).describe()

,ENDMARKETINGDATE
count,6.456000e+03
mean,2.020787e+07
std,1.055415e+04
min,2.020022e+07
25%,2.020080e+07
50%,2.020123e+07
75%,2.021111e+07
max,2.039083e+07


In [17]:
package_df.select_dtypes(include=['int64']).describe()

,STARTMARKETINGDATE
count,1.738870e+05
mean,2.012930e+07
std,3.844789e+05
min,1.900010e+07
25%,2.010060e+07
50%,2.015110e+07
75%,2.018050e+07
max,2.997122e+07


In [18]:
package_df.select_dtypes(include=['object']).describe()

,PRODUCTID,PRODUCTNDC,NDCPACKAGECODE,PACKAGEDESCRIPTION,NDC_EXCLUDE_FLAG,SAMPLE_PACKAGE
count,173887,172387,171541,173887,173887,173887
unique,93084,91080,171447,173885,1,2
top,73069-525_e338a0c8-2ca2-42a4-8e4d-d76b4002ee48,73069-095,20211201,"750 mL in 1 BOTTLE, PLASTIC (47593-359-41)",N,N
freq,72,72,43,2,173887,173223


#### Nombre, taux et types des valeurs manquantes par attribut (package_df)

In [19]:
table_completeness(package_df)

,Types,Nb Unique,Nb manquants,Ratio manquants%
PRODUCTID,object,93084,0,0.000000
PACKAGEDESCRIPTION,object,173885,0,0.000000
STARTMARKETINGDATE,int64,7401,0,0.000000
NDC_EXCLUDE_FLAG,object,1,0,0.000000
SAMPLE_PACKAGE,object,2,0,0.000000
PRODUCTNDC,object,91080,1500,0.008626
NDCPACKAGECODE,object,171447,2346,0.013492
ENDMARKETINGDATE,float64,767,167431,0.962872


_____________

# 2.

Listez toutes les relations/règles observées entre les attributs (informations communes, corrélations, chaînes de caractéres communes, attribut inclus dans un autre, ordre des valeurs) ;


## Informations Communes

* **ProductID**, **ProductNDC**, **StartMarketingDate**, **EndMarketingDate**, **NDC_Exclude_Flag** (_product_ & _package_) - Ces variables sont partagées par les deux tables. Selon la documentation, il est préférable de joindre les tables par l'entremise de la variable __ProductID__ considérant que celle-ci a été ajustée pour empêcher la duplication de ligne. 


## Corrélations

* **Pharm_Classes** - Cette variable devrait démontrer une corrélation avec la variable __SubstanceName__ puisque la documentation les décris comme étant reliées;

* **DosageFormName** & **RouteName** - Ceux-ci devraient démontrer une corrélation puisque l'un (__DosageFormName__) représente les modalités d'administration des médicaments et l'autre (__RouteName__) représente textuellement la manière dont le médicament sera transmis;

* Considérant les informations spécifiques au contenant inclues dans __PackageDescription__ (_package_), il est envisageable que ceux-ci soient corrélés aux __DosageFormName__ et __RouteName__ qui ont trait à la manière d'administrer le médicament.


## Chaines de caractères communes

* __PackageDescription__ (_package_) - Cette variable implique plusieurs informations sous la forme d'une phrase. On retrouve notamment l'information de __NDCPackageCode__ ainsi que des informations d'unités de mesure et des types de contenant.


## Attribut inclus dans un autre

* **ProductID** (_product_ & _package_) - Implique la concaténation de __ProductNDC__ et un identifiant _SPL_. Cette combinaison permet de prévenir la duplication de lignes lorsque les deux tables seront jointes. 
 
* **ApplicationNumber** (_product_) - Représente majoritairement la concaténation entre __MarketingCategoryName__ et ce qui semble être le numéro de série associé au produit;

* __NDCPackageCode__ (_package_) - Contient les informations de la variable __ProductNDC__ ainsi qu'un code représentant le type de paquet utilisé; cette dernière information pourrait être pertinente;


## Ordre des valeurs

* **StrengthNumber** & **StrengthUnit** (_product_) - Il est à considérer que l'utilisation des valeurs de la variable __StrenghtNumber__ doivent être joint de __StrenghtUnit__. N'ayant pas tous la même unité de mesures, ceux-ci devront être convertis sous une seule et même unité de mesure pour fin de bonne analyse.
 
 



_____________

# 3.

Détectez et corrigez les incohérences entre des valeurs d’attributs dans les deux tables ; pour
chaque règle identifiée à la question précédente, détectez et corrigez les cas où la règle n’est
pas respectée ;

### 3.1
* Les valeurs des attributs __StartMarketingDate__, __EndMarketingDate__ et __Listing_Record_Certified_Through__ peuvent être transformées en format _date_.


In [20]:
product_df['STARTMARKETINGDATE'] = pd.to_datetime(product_df['STARTMARKETINGDATE'], format='%Y%m%d', errors='coerce')
product_df['ENDMARKETINGDATE'] = pd.to_datetime(product_df['ENDMARKETINGDATE'], format='%Y%m%d', errors='coerce')
product_df['LISTING_RECORD_CERTIFIED_THROUGH'] = pd.to_datetime(product_df['LISTING_RECORD_CERTIFIED_THROUGH'], format='%Y%m%d', errors='coerce')

* Confirmation qu'il n'y a pas de date d'échance si le produit à atteint ca date de fin de marketing (ENDMARKETINGDATE)

In [21]:
# Si le produit est venu à échéance, il ne devrait pas y avoir de valeur pour la variable LISTING_RECORD_CERTIFIED_THROUGH
print("(product) Nombre de produits ayant atteint leur date de fin affichant toujours une date d'expiration: {0}".format(product_df[product_df['ENDMARKETINGDATE'].notna()]['LISTING_RECORD_CERTIFIED_THROUGH'].notna().sum()))

(product) Nombre de produits ayant atteint leur date de fin affichant toujours une date d'expiration: 0


* Confirmation qu'il n'y a pas de date de début de market (STARTMARKETINGDATE) qui serait après la date de fin d'expiration (LISTING_RECORD_CERTIFIED_THROUGH)

In [22]:
(product_df['STARTMARKETINGDATE'] > product_df['LISTING_RECORD_CERTIFIED_THROUGH']).sum()

0

In [23]:
package_df['STARTMARKETINGDATE'] = pd.to_datetime(package_df['STARTMARKETINGDATE'], format='%Y%m%d', errors='coerce')
package_df['ENDMARKETINGDATE'] = pd.to_datetime(package_df['ENDMARKETINGDATE'], format='%Y%m%d', errors='coerce')

#### Confirmation :
* Date de début (STARTMARKETINGDATE)  <  Date de fin de marketing (ENDMARKETINGDATE)
* Date de début (STARTMARKETINGDATE)  <  Date d'expiration (LISTING_RECORD_CERTIFIED_THROUGH)}

In [24]:
# La date de début de marketing (STARTMARKETINGDATE) devrait précéder la date de fin de production (ENDMARKETINGDATE) 
# ou celle d'expiration (LISTING_RECORD_CERTIFIED_THROUGH).
print("(product) Nombre de produits dont la date de début (STARTMARKETINGDATE) est après sa date de fin (ENDMARKETINGDATE): {0}".format((product_df['STARTMARKETINGDATE'] > product_df['ENDMARKETINGDATE']).sum()))
print("(product) Nombre de produits dont la date de début (STARTMARKETINGDATE) est après sa date d'expiration (LISTING_RECORD_CERTIFIED_THROUGH): {0}".format((product_df['STARTMARKETINGDATE'] > product_df['LISTING_RECORD_CERTIFIED_THROUGH']).sum()))
print("(package) Nombre de produits dont la date de début (STARTMARKETINGDATE) est après sa date de fin (ENDMARKETINGDATE): {0}".format((package_df['STARTMARKETINGDATE'] > package_df['ENDMARKETINGDATE']).sum()))


(product) Nombre de produits dont la date de début (STARTMARKETINGDATE) est après sa date de fin (ENDMARKETINGDATE): 0
(product) Nombre de produits dont la date de début (STARTMARKETINGDATE) est après sa date d'expiration (LISTING_RECORD_CERTIFIED_THROUGH): 0
(package) Nombre de produits dont la date de début (STARTMARKETINGDATE) est après sa date de fin (ENDMARKETINGDATE): 0


### 3.2
* Les variables _ProductId_, _ProductNDC_ et NDCPackageCode suivent un format précis. Ceux-ci doivent être analysés afin de déterminer s'il y a des formats non acceptables.

    - PRODUCTNDC doit répondre à une structure de digits telle que {3-5}, {3-4}, {4-4}, {4-5}.
    - PRODUCTID concatène la valeur du PRODUCTNDC et un identifiant SPL séparé par un '_'.
    - NDCPACKAGECODE concatène la valeur du PRODUCTNDC et un code segment de 2 digits séparé par '-'.


In [25]:
ndc_pkg_code_wrong = package_df['NDCPACKAGECODE'].str.split('-').apply(lambda x: len(x) if isinstance(x, list) else 0).copy()
ndc_pkg_code_wrong_index = ndc_pkg_code_wrong[(ndc_pkg_code_wrong < 3) & (ndc_pkg_code_wrong > 0)].index

In [26]:
print("Il y a un total de {0} valeurs incohérentes pour la variable NDCPACKAGECODE de la table package".format(package_df.loc[ndc_pkg_code_wrong_index, 'NDCPACKAGECODE'].shape[0]))

Il y a un total de 154 valeurs incohérentes pour la variable NDCPACKAGECODE de la table package


Puisque nous traitons les valeurs manquantes dans la prochaine section et qu'il est possible de déterminer le _NDCPackageCode_ à l'aide de la variable _PackageDescription_, la valeur NaN sera attribué à ces valeurs incohérentes.

In [27]:
package_df.loc[ndc_pkg_code_wrong_index, 'NDCPACKAGECODE'] = np.nan

### 3.3

* La variable _ApplicationNumber_ suit un format particulier en fonction des valeurs de la colonne _MarketingCategoryName_
    * Tel qu'il est mentionné dans la documentation, si la valeur sous _MarketingCategoryName_ représente _OTC Monograph Final_ ou _OTC Monograph Not Final_, la valeur de _ApplicationNumber_ représente la citation CFR (e.i. "part341")

On remarque que certaines valeurs ne suivent pas le format. Nous effectuerons les modifications suivantes en fonction du cas:
* s'il n'y a que le numéros (e.i. "341") -> nous ajoutons "part" en préfixe
* s'il n'y a que part -> nous modifions pour NaN
* s'il y a plus d'une valeur, nous garderons que la première référence puisque l'attribut _ApplicationNumber_ n'est pas identifié _MV_ (multiple value; voir documentation)

In [28]:
# Il semble y avoir mauvaise attribution de la nomenclature
set(product_df[product_df['MARKETINGCATEGORYNAME'].isin(['OTC MONOGRAPH FINAL', 'OTC MONOGRAPH NOT FINAL'])]['APPLICATIONNUMBER'])

{'333D',
 'part',
 'part331',
 'part332',
 'part333',
 'part333A',
 'part333B',
 'part333C',
 'part333D',
 'part333E',
 'part334',
 'part335',
 'part336',
 'part338',
 'part340',
 'part341',
 'part341,part348',
 'part343',
 'part344',
 'part346',
 'part347',
 'part348',
 'part349',
 'part349B',
 'part350',
 'part352',
 'part355',
 'part355B',
 'part356',
 'part356,part355',
 'part357',
 'part357B',
 'part357I',
 'part358',
 'part358A',
 'part358B',
 'part358D',
 'part358F',
 'part358G',
 'part358H'}

In [29]:
# add "part" to actual number
product_df['APPLICATIONNUMBER'] = np.where(
    (product_df['MARKETINGCATEGORYNAME'].isin(['OTC MONOGRAPH FINAL', 'OTC MONOGRAPH NOT FINAL'])) &
    (~product_df['APPLICATIONNUMBER'].str.contains("part", na=False)),
    "part" + product_df['APPLICATIONNUMBER'],
    product_df['APPLICATIONNUMBER'])

In [30]:
# change to NaN if value == "part"
product_df.loc[product_df['APPLICATIONNUMBER'] == 'part', 'APPLICATIONNUMBER'] = np.nan

In [31]:
# remove multiple value in the same cell
product_df['APPLICATIONNUMBER'] = np.where(
    product_df['MARKETINGCATEGORYNAME'].isin(['OTC MONOGRAPH FINAL', 'OTC MONOGRAPH NOT FINAL']),
    product_df['APPLICATIONNUMBER'].str.split(',').str[0],
    product_df['APPLICATIONNUMBER'])

### Confirmation que les produits non approuvés n'ont pas de numéro d'application associé.

In [32]:
product_df[product_df['MARKETINGCATEGORYNAME'].isin(
    ['UNAPPROVED DRUG FOR USE IN DRUG SHORTAGE',
     'UNAPPROVED DRUG OTHER',
     'UNAPPROVED HOMEOPATHIC',
     'UNAPPROVED MEDICAL GAS'])]['APPLICATIONNUMBER'].notna().sum()

0

### Confirmation Valeurs Multiple

SUBSTANCENAME, ACTIVE_NUMERATOR_STRENGTH, ACTIVE_INGRED_UNIT sont des variables à valeurs multiples. Celles-ci, lorsque multiple, sont listées dans le même ordre.

Confirmation du même nombre d'éléments pour chacune de ces variables

In [33]:
mv_errors = []
for index, row in product_df.iterrows():
    #row['SUBSTANCENAME'].str.split(';'). == row['ACTIVE_NUMERATOR_STRENGTH'] == row['ACTIVE_INGRED_UNIT']
    _name_count = len(row['SUBSTANCENAME'].split(';')) if isinstance(row['SUBSTANCENAME'], str) else 0
    _strength_count = len(row['ACTIVE_NUMERATOR_STRENGTH'].split(';')) if isinstance(row['ACTIVE_NUMERATOR_STRENGTH'], str) else 0
    _unit_count = len(row['ACTIVE_INGRED_UNIT'].split(';')) if isinstance(row['ACTIVE_INGRED_UNIT'], str) else 0

    if _name_count == _strength_count == _unit_count:
        continue
    else:
        mv_errors.append(index)


#### Explication

On constate que dans ce cas, ce n'est que l'élément du split qui se retrouve à l'intérieur de parenthèses. Nous pouvons donc ignorer cette alerte.

In [34]:
mv_errors

[90536]

In [35]:
print("Original: ", product_df.loc[90536, 'SUBSTANCENAME'])
print("Split: ", product_df.loc[90536, 'SUBSTANCENAME'].split(';'))

Original:  GLYCERIN; HYDROLYZED SOY PROTEIN (ENZYMATIC; 2000 MW)
Split:  ['GLYCERIN', ' HYDROLYZED SOY PROTEIN (ENZYMATIC', ' 2000 MW)']


In [36]:
print("Original: ", product_df.loc[90536, 'ACTIVE_NUMERATOR_STRENGTH'])
print("Split: ", product_df.loc[90536, 'ACTIVE_NUMERATOR_STRENGTH'].split(';'))

Original:  10; .12
Split:  ['10', ' .12']


In [37]:
print("Original: ", product_df.loc[90536, 'ACTIVE_INGRED_UNIT'])
print("Split: ", product_df.loc[90536, 'ACTIVE_INGRED_UNIT'].split(';'))

Original:  g/100g; g/100g
Split:  ['g/100g', ' g/100g']


_______

## 4.

Complétez au maximum les données manquantes dans les deux tables ;

### Product

In [38]:
# Complète la variable PRODUCTID à l'aide d'un mapping provenant de la table package

prodIdMissing_mask = product_df['PRODUCTID'].isna()
packageID_map = package_df[['PRODUCTID', 'PRODUCTNDC']].set_index('PRODUCTNDC').drop_duplicates()

# Apply mapping from package_df
product_df.loc[prodIdMissing_mask, 'PRODUCTID'] = product_df.loc[prodIdMissing_mask, 'PRODUCTNDC'].map(
    packageID_map['PRODUCTID'].to_dict()).values

### Package

In [39]:
pkg_mask = package_df['NDCPACKAGECODE'].isna()
package_df.loc[pkg_mask, 'NDCPACKAGECODE'] = package_df.loc[pkg_mask, 'PACKAGEDESCRIPTION'].str.extract(r'\((.*?)\)').values

In [40]:
prod_mask = package_df['PRODUCTNDC'].isna()
package_df.loc[prod_mask, 'PRODUCTNDC'] = package_df['NDCPACKAGECODE'].str.split('-').apply(lambda x: '-'.join(x[:2]))

_____________________

## 5.

Détectez et retirez les objets dupliqués dans les deux tables ;

### Possibilité de données dupliquées

#### Product

* PRODUCTID - Doit être unique puisqu'il représente le produit dans la table _product_

#### Package

* NDCPACKAGECODE - Doit être unique puisqu'il représente le paquet dans la table _package_

In [41]:
print("Nombre de code produit en double: ", product_df['PRODUCTID'].duplicated().sum())
print("Nombre de code paquet en double: ", package_df['NDCPACKAGECODE'].duplicated().sum())


Nombre de code produit en double:  0
Nombre de code paquet en double:  2


In [42]:
duplicated_pkg_code = package_df[package_df['NDCPACKAGECODE'].duplicated()]['NDCPACKAGECODE'].values
first_duplicated = package_df[package_df['NDCPACKAGECODE'] == duplicated_pkg_code[0]].copy()
second_duplicated = package_df[package_df['NDCPACKAGECODE'] == duplicated_pkg_code[1]].copy()

#### Premier doublons

In [43]:
first_duplicated

,PRODUCTID,PRODUCTNDC,NDCPACKAGECODE,PACKAGEDESCRIPTION,STARTMARKETINGDATE,ENDMARKETINGDATE,NDC_EXCLUDE_FLAG,SAMPLE_PACKAGE
53622,45802-929_4290e001-c03c-4bde-a132-5203cc57afb4,45802-929,45802-929-49,4 CARTON in 1 KIT (45802-929-49) > 1 KIT in 1...,2018-08-02,NaT,N,N
53623,45802-929_b2a5b110-7537-4ffc-bf0a-f0fe13379c9d,45802-929,45802-929-49,4 CARTON in 1 KIT (45802-929-49) > 1 KIT in 1...,2018-08-02,NaT,N,N


On remarque qu'il y a différence au niveau du _PRODUCTID_. Ceux-ci semblent faire référence à deux produits différents. <br>


In [44]:
product_df[product_df['PRODUCTID'].isin(first_duplicated['PRODUCTID'])]

,PRODUCTID,PRODUCTNDC,PRODUCTTYPENAME,PROPRIETARYNAME,PROPRIETARYNAMESUFFIX,NONPROPRIETARYNAME,DOSAGEFORMNAME,ROUTENAME,STARTMARKETINGDATE,ENDMARKETINGDATE,MARKETINGCATEGORYNAME,APPLICATIONNUMBER,LABELERNAME,SUBSTANCENAME,ACTIVE_NUMERATOR_STRENGTH,ACTIVE_INGRED_UNIT,PHARM_CLASSES,DEASCHEDULE,NDC_EXCLUDE_FLAG,LISTING_RECORD_CERTIFIED_THROUGH
31495,45802-929_4290e001-c03c-4bde-a132-5203cc57afb4,45802-929,HUMAN PRESCRIPTION DRUG,Mesalamine,NaN,Mesalamine,KIT,NaN,2018-08-02,NaT,ANDA,ANDA076751,Perrigo New York Inc,NaN,NaN,NaN,NaN,NaN,N,2020-12-31
31496,45802-929_b2a5b110-7537-4ffc-bf0a-f0fe13379c9d,45802-929,HUMAN PRESCRIPTION DRUG,Mesalamine,NaN,Mesalamine,KIT,NaN,2018-08-02,NaT,ANDA,ANDA076751,Perrigo New York Inc,NaN,NaN,NaN,NaN,NaN,N,2020-12-31


On remarque qu'à l'exception du _PRODUCTID_, cela semble être le même produit. De ce fait, nous allons en retirer un dans la table _product_ et son équivalent dans la table _package_.

In [45]:
first_removed_duplicated_id = first_duplicated['PRODUCTID'].values[1]

In [46]:
product_df = product_df[product_df['PRODUCTID'] != first_removed_duplicated_id]
package_df = package_df[package_df['PRODUCTID'] != first_removed_duplicated_id]

#### Deuxième doublons

In [47]:
second_duplicated

,PRODUCTID,PRODUCTNDC,NDCPACKAGECODE,PACKAGEDESCRIPTION,STARTMARKETINGDATE,ENDMARKETINGDATE,NDC_EXCLUDE_FLAG,SAMPLE_PACKAGE
56138,47593-359_a406ac88-de15-4494-88e6-bd0f3dc77793,47593-359,47593-359-41,"750 mL in 1 BOTTLE, PLASTIC (47593-359-41)",2002-07-26,NaT,N,N
56139,47593-359_a406ac88-de15-4494-88e6-bd0f3dc77793,47593-359,47593-359-41,"750 mL in 1 BOTTLE, PLASTIC (47593-359-41)",2013-09-05,NaT,N,N


On remarque qu'il y a différence au niveau du _STARTMARKETINGDATE_. Ceux-ci font référence à deux dates différentes. Afin de confirmer laquelle des lignes est la bonne, nous allons voir dans la table _product_ pour confirmer quelle date est la bonne.

In [48]:
print("La date de référence est le {0}".format(str(product_df[product_df['PRODUCTID'].isin(second_duplicated['PRODUCTID'])]['STARTMARKETINGDATE'].values[0])))

La date de référence est le 2002-07-26T00:00:00.000000000


De ce fait, nous allons retirer la ligne faisant référence à l'année de départ 2013-09-05

In [49]:
package_df = package_df[package_df['STARTMARKETINGDATE'] != second_duplicated['STARTMARKETINGDATE'].values[1]]

## 6.

Intégrez les deux tables et nettoyez le résultat (données dupliquées, incomplètes, incohérentes, erronées) ;


In [50]:
merged_table = pd.merge(product_df, package_df, on='PRODUCTID', suffixes=('_product', '_package'))

In [51]:
table_completeness(merged_table)

,Types,Nb Unique,Nb manquants,Ratio manquants%
PRODUCTID,object,93079,0,0.000000
PACKAGEDESCRIPTION,object,173876,0,0.000000
NDCPACKAGECODE,object,173876,0,0.000000
PRODUCTNDC_package,object,91693,0,0.000000
NDC_EXCLUDE_FLAG_product,object,1,0,0.000000
NDC_EXCLUDE_FLAG_package,object,1,0,0.000000
LABELERNAME,object,6603,0,0.000000
MARKETINGCATEGORYNAME,object,10,0,0.000000
SAMPLE_PACKAGE,object,2,0,0.000000
DOSAGEFORMNAME,object,134,0,0.000000


On remarque qu'il y a 4 variables traitant des mêmes sujets dans les deux tables:

Nous déterminerons les variables en fonction de leur complétude vis-à-vis la table intégrée.

* NDC_EXCLUDE_FLAG - Aucune valeur manquante. Nous en garderons simplement une;
* PRODUCTNDC - Aucune valeur manquante. Nous en garderons simplement une;
* STARTMARKETINGDATE - On constate que la variable provenant de la table _package_ affiche des valeurs manquantes. Nous retirerons donc *STARTMARKETINGDATE_package*;
* ENDMARKETINGDATE - On constate qu'il y a moins de données manquantes provenant de la table _package_. Nous retirerons donc *ENDMARKETINGDATE_product*;

In [52]:
merged_table.drop(columns=['PRODUCTNDC_package', 'NDC_EXCLUDE_FLAG_package',
                           'STARTMARKETINGDATE_package', 'ENDMARKETINGDATE_product'],
                  inplace=True)

merged_table.rename(columns={'PRODUCTNDC_product': 'PRODUCTNDC',
                             'NDC_EXCLUDE_FLAG_product': 'NDC_EXCLUDE_FLAG',
                             'STARTMARKETINGDATE_product': 'STARTMARKETINGDATE',
                             'ENDMARKETINGDATE_package': 'ENDMARKETINGDATE'}, inplace=True)

______________________

## 7.

Proposez un nouvel ensemble d’attributs (représentation) qui élimine la redondance des informations dans les valeurs des attributs, et qui permet de transformer l’attribut PHARM_CLASSES
en un ensemble d’attributs distincts correspondant à ses différents champs EPC, CS, MOA,
PE etc. ;


In [53]:
merged_table['PHARM_CLASSES'].head()

0                                                  NaN
1    Radioactive Diagnostic Agent [EPC],Positron Em...
2    Radioactive Diagnostic Agent [EPC],Positron Em...
3    GLP-1 Receptor Agonist [EPC],Glucagon-Like Pep...
4    GLP-1 Receptor Agonist [EPC],Glucagon-Like Pep...
Name: PHARM_CLASSES, dtype: object

In [54]:
notna_mask = merged_table['PHARM_CLASSES'].notna()

EPC_mask = np.where(notna_mask & merged_table['PHARM_CLASSES'].str.contains('[EPC]'))
CS_mask = np.where(notna_mask & merged_table['PHARM_CLASSES'].str.contains('[CS]'))
MoA_mask = np.where(notna_mask & merged_table['PHARM_CLASSES'].str.contains('[MoA]'))
PE_mask = np.where(notna_mask & merged_table['PHARM_CLASSES'].str.contains('[PE]'))


In [55]:
merged_table['EPC'] = merged_table.loc[EPC_mask, 'PHARM_CLASSES'].str.split(',').apply(
    lambda x: [s for s in x if "EPC" in s]).str[0]

merged_table['CS'] = merged_table.loc[CS_mask, 'PHARM_CLASSES'].str.split(',').apply(
    lambda x: [s for s in x if "CS" in s]).str[0]

merged_table['MoA'] = merged_table.loc[MoA_mask, 'PHARM_CLASSES'].str.split(',').apply(
    lambda x: [s for s in x if "MoA" in s]).str[0]

merged_table['PE'] = merged_table.loc[PE_mask, 'PHARM_CLASSES'].str.split(',').apply(
    lambda x: [s for s in x if "PE" in s]).str[0]

In [56]:
merged_table[['EPC', 'CS', 'MoA', 'PE']].head()

,EPC,CS,MoA,PE
0,NaN,NaN,NaN,NaN
1,Radioactive Diagnostic Agent [EPC],NaN,Positron Emitting Activity [MoA],NaN
2,Radioactive Diagnostic Agent [EPC],NaN,Positron Emitting Activity [MoA],NaN
3,GLP-1 Receptor Agonist [EPC],Glucagon-Like Peptide 1 [CS],Glucagon-like Peptide-1 (GLP-1) Agonists [MoA],NaN
4,GLP-1 Receptor Agonist [EPC],Glucagon-Like Peptide 1 [CS],Glucagon-like Peptide-1 (GLP-1) Agonists [MoA],NaN


_____________________

## 8.

À partir de la nouvelle représentation, proposez un ensemble d’attributs à utiliser pour
prédire le plus précisément possible toutes les classes pharmacologiques établies d’un médicament (champ EPC dans l’attribut PHARM_CLASSES) ;


### Attributs à considérer

* SUBSTANCENAME - Tel que mentionné, il semble y avoir une forte corrélation entre SUBSTANCENAME et la variable PHARM_CLASSES
* ROUTENAME, DOSAGEFORMNAME - ROUTENAME et DOSAGEFORMNAME devrait aussi démontrer une corrélation puisque ceux-ci représente les modalités d'administration des médicaments et la forme de dosage
* MARKETINGCATEGORYNAME - Celle-ci pourrait être informative puisqu'elle représente la catégorie de marketing qui devrait diriger vers des classes pharmaceutiques.

### Attributs à ignorer

* PRODUCTID, PRODUCTNDC, PRODUCTTYPENAME
* PROPRIETARYNAME, PROPRIETARYNAMESUFFIX
* NONPROPRIETARYNAME
* STARTMARKETINGDATE, ENDMARKETINGDATE
* APPLICATIONNUMBER
* LABELERNAME 
* ACTIVE_NUMERATOR_STRENGTH, ACTIVE_INGRED_UNIT,
* DEASCHEDULE
* NDC_EXCLUDE_FLAG
* NDCPACKAGECODE, PACKAGEDESCRIPTION, SAMPLE_PACKAGE

In [57]:
merged_table.columns

Index(['PRODUCTID', 'PRODUCTNDC', 'PRODUCTTYPENAME', 'PROPRIETARYNAME',
       'PROPRIETARYNAMESUFFIX', 'NONPROPRIETARYNAME', 'DOSAGEFORMNAME',
       'ROUTENAME', 'STARTMARKETINGDATE', 'MARKETINGCATEGORYNAME',
       'APPLICATIONNUMBER', 'LABELERNAME', 'SUBSTANCENAME',
       'ACTIVE_NUMERATOR_STRENGTH', 'ACTIVE_INGRED_UNIT', 'PHARM_CLASSES',
       'DEASCHEDULE', 'NDC_EXCLUDE_FLAG', 'LISTING_RECORD_CERTIFIED_THROUGH',
       'NDCPACKAGECODE', 'PACKAGEDESCRIPTION', 'ENDMARKETINGDATE',
       'SAMPLE_PACKAGE', 'EPC', 'CS', 'MoA', 'PE'],
      dtype='object')

In [58]:
attribut_predict = ['SUBSTANCENAME', 'ROUTENAME', 'DOSAGEFORMNAME', 'MARKETINGCATEGORYNAME']

Nous pouvons utiliser la méthode de Cramers V pour nous donner une mesure de similarité avec la variable d'étude soit _EPC_

In [59]:
for _ in attribut_predict:
    print("Attribut [{0}] Cramers V score: {1}".format(_, cramers_v(merged_table[_], merged_table['EPC'])))

Attribut [SUBSTANCENAME] Cramers V score: 0.9854850613909063
Attribut [ROUTENAME] Cramers V score: 0.3380114278587186
Attribut [DOSAGEFORMNAME] Cramers V score: 0.3455459092352666
Attribut [MARKETINGCATEGORYNAME] Cramers V score: 0.5390173211993291


________________

## 9.

En se basant sur la réduction de dimension obtenue à la question précédente, appliquez un
modèle de classification pour prédire les classes pharmacologiques établies des médicaments
pour lesquels l’information est manquante ;

Après avoir réduit les dimenstions, il est nécessaire de tranformer les données qualitatives en données
quantitative afin d'être en mesure d'utiliser les alogrithme de _clustering_. À cet effet, on utilise le
_LabelEncoder_ de _Sci-Kit learn_:

In [141]:
enc = LabelEncoder()
enc_epc = LabelEncoder()
enc_cs = LabelEncoder()
enc_moa = LabelEncoder()
enc_pe = LabelEncoder()

# Save the indexes for which we have at least one PHARM_CLASS value, those will be used for training
training_indexes = np.array(merged_table['EPC'].notnull() |
                            merged_table['CS'].notnull() |
                            merged_table['MoA'].notnull() |
                            merged_table['PE'].notnull())

# Save the indexes for which no PHARM_CLASS value at all, those will be predicted after the training
prediction_indexes = np.array(merged_table['EPC'].isnull() &
                              merged_table['CS'].isnull() &
                              merged_table['MoA'].isnull() &
                              merged_table['PE'].isnull())

Xs = np.array([enc.fit_transform(_) for _ in merged_table[attribut_predict].T.values]).T

X = Xs[training_indexes]

# ys = np.array([enc_epc.fit_transform(merged_table['EPC'].values),
#               enc_cs.fit_transform(merged_table['CS'].values),
#               enc_moa.fit_transform(merged_table['MoA'].values),
#               enc_pe.fit_transform(merged_table['PE'].values)]).T
#
# y = ys[training_indexes]

ys = enc_epc.fit_transform(merged_table['EPC'].values).T
y = ys[training_indexes]

Une fois les variables encoder, on peut entrainer nos modèles sur un sous-ensemble des données qui contiennent au moins
une valeur de PHARM_CLASS:

In [142]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

clfs = {'KNeighborsClassifier': KNeighborsClassifier(n_neighbors=5),
        'RandomForestClassifier':  RandomForestClassifier(max_depth=3, random_state=0)}

best_model = {'name': '', 'score': 0, 'model': None}

for name, clf in clfs.items():
    print(f'Modèle {name}')
    print('## Entrainement ##')
    # classifier = MultiOutputClassifier(clf, n_jobs=1)
    classifier = clf
    classifier.fit(X_train, y_train)
    train_score = classifier.score(X_train, y_train)
    print(f'Score d\'entraînement: {train_score}')

    print('## Test ##')
    test_predictions = classifier.predict(X_test)
    test_score = classifier.score(X_test, y_test)
    print(f'Score de test: {test_score}')

    if test_score > best_model.get('score'):
        best_model['name'], best_model['score'], best_model['model'] = name, test_score, clf

print(f"Le modèle présentant le meilleur score est {best_model.get('name')} avec {best_model.get('score')}")

Modèle KNeighborsClassifier
## Entrainement ##
Score d'entraînement: 0.9807705114544486
## Test ##
Score de test: 0.9686654948620876
Modèle RandomForestClassifier
## Entrainement ##
Score d'entraînement: 0.13057738412360148
## Test ##
Score de test: 0.1304421308815576
Le modèle présentant le meilleur score est KNeighborsClassifier avec 0.9686654948620876


On constate donc une bonne performance de la part du classificateur KNN. On peut ainsi procéder à la classification:

In [144]:
test_predictions = classifier.predict(Xs[prediction_indexes])

pred_epc = enc_epc.inverse_transform(test_predictions)

predictions = merged_table[prediction_indexes]
predictions['EPC'] = enc_epc.inverse_transform(test_predictions)

Voici un aperçu du résultat:

In [145]:
predictions[['SUBSTANCENAME']].head()

,SUBSTANCENAME,EPC,CS,MoA,PE
0,WATER,Corticosteroid [EPC],NaN,NaN,NaN
7,GALCANEZUMAB,Non-Standardized Pollen Allergenic Extract [EPC],NaN,NaN,NaN
8,GALCANEZUMAB,Non-Standardized Pollen Allergenic Extract [EPC],NaN,NaN,NaN
13,GALCANEZUMAB,Non-Standardized Pollen Allergenic Extract [EPC],NaN,NaN,NaN
14,GALCANEZUMAB,Non-Standardized Pollen Allergenic Extract [EPC],NaN,NaN,NaN
